In [116]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
import pickle

In [108]:
def read_dataframe(file_name):
    df = pd.read_parquet(file_name)
    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >=1) & (df.duration <=60)]
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype('str')
    return df 


In [109]:
df_train = read_dataframe("../data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("../data/green_tripdata_2021-02.parquet")

In [110]:
df_train['PU_DO'] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val['PU_DO'] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

In [111]:
categorical = ["PU_DO"] #['PULocationID', 'DOLocationID']
numerical = ['trip_distance']


dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dict = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dict)


In [112]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [113]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.758715206931833

In [114]:
lasso = Lasso(alpha=0.001)
lasso.fit(X_train, y_train)

y_pred = lasso.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

9.233436225720547

In [115]:
ridge = Ridge(alpha=0.001)
ridge.fit(X_train, y_train)

y_pred = ridge.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.511008883160363

In [117]:
with open("../models/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

In [118]:
with open("../models/lasso_reg.bin", "wb") as f_out:
    pickle.dump((dv, lasso), f_out)